In [ ]:
import scanpy as sc
import os
import numpy as np
from tqdm.notebook import tqdm
import glob
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import scvi

In [2]:
input_folders = glob.glob("uninfected/segmentation_SI*")

In [ ]:
adatas = []
for input_file in input_folders:
    adatas_ = sc.read(os.path.join(input_file, "adatas", "09_before_decomposition_model.h5ad"))
    adatas_.obs['batch'] = input_file.split("/")[-1]
    
    original_adata = sc.read(os.path.join(input_file, "adatas", "04_tissue_cleared.h5ad"))
    original_adata.obs = adatas_.obs
    original_adata.obsm = adatas_.obsm
    original_adata.uns = adatas_.uns
    adatas.append(original_adata)
adatas = sc.concat(adatas)

In [17]:
adatas.obs = adatas.obs.drop(
    adatas.obs.columns[adatas.obs.columns.str.contains("Topic ")], axis=1
)

Write out combined object to an output folder

In [19]:
output_folder = r"/mnt/sata1/Analysis_Alex/uninfected"

In [21]:
adatas.write(os.path.join(output_folder, "integrated_celltyped.h5ad"))

In [ ]:
concatenated_all = sc.read(os.path.join(output_folder, "integrated_celltyped.h5ad"))

### We also want to clip our epithelial values at 0.6 for visualization

In [23]:
concatenated_all.obs["epithelial_distance_clipped"] = np.clip(
    concatenated_all.obs["epithelial_distance"].values, 0, 0.6
)

### Write out the final object

In [24]:
concatenated_all.write(os.path.join(output_folder, "final_celltyped_and_axes.h5ad"))